In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('wind turbine inventory.xlsx')
df

30kW  \
Rotor Materials       Fiberglass                                0.402204   
                      Chromium steel                            0.380165   
                      Cast iron                                 0.217631   
Rotor Disposal        Iron                                      0.217631   
                      Steel                                     0.380165   
                      Polymer                                   0.140496   
                      Glass                                     0.261708   
Nacelle Materials     Low-alloy steel                           0.052615   
                      Chromium steel                            0.642596   
                      Cast iron                                 0.202942   
                      Rubber                                    0.003382   
                      Aluminium                                 0.015784   
                      Copper                                    0.018254   
                      Fiberglass                                0.064426   
                      Lubricating oil                           0.010738   
Nacelle Disposal      Steel                                     0.695211   
                      Iron                                      0.202942   
                      Aluminium                                 0.015784   
                      Copper                                    0.018254   
                      Used oil                                  0.010738   
                      Polymer                                   0.025931   
                      Glass                                     0.041877   
Nacelle Assembly      Copper wire drawing                       0.018254   
                      Aluminium sheet rolling                   0.015784   
                      Chromium steel sheet rolling (Rotor)      0.380165   
                      Chromium steel sheet rolling (Nacelle)    0.642596   
                      Steel sheet rolling (Rotor)               0.217631   
                      Steel sheet rolling (Nacelle)             0.255557   
                      Electricity [kWh]                       575.000000   
Tower Materials       Low-alloy steel                           0.993066   
...                                                                  ...   
Power Supply          Copper                                    0.356564   
                      HDPE granules                             0.354943   
                      PP granules                               0.032415   
                      PVC impact resistant                      0.256078   
Powe Supply Assembly  Copper wire drawing                       0.356564   
Power Supply Disposal Copper                                    0.356564   
                      HDPE granules                             0.354943   
                      PVC impact resistant                      0.256078   
                      PP granules                               0.032415   
Transformer           Total weight (kg)                              NaN   
                      Electric steel                                 NaN   
                      Construction steel                             NaN   
                      Transformer oil                                NaN   
                      Aluminium                                      NaN   
                      Insulation material                            NaN   
                      Porcelain                                      NaN   
                      Copper                                         NaN   
                      Paint                                          NaN   
                      Wood                                           NaN   
Transformer Assembly  Copper wire drawing                            NaN   
                      Steel sheet rolling                            NaN   
                      Aluminium sheet ro

In [3]:
# there are five detailed life cycle inventories of wind turbines, their rated powers are 30 kW, 150 kW, 600 kW, 800 kW and 2000 kW
# After we know the mass of a wind turbine component, then interpolation, regression or average values are applied to get the split ratio
# and then get the amount of specific materials

def get_materials_rotor(p,rotor_mass): # get materials of rotor, p is rated power
    rated_power = [30,150,600,800,2000]
    materials = {}
    disposal = {}
    
    for item in list(df.loc['Rotor Materials'].index):
        split_ratio = list(df.loc['Rotor Materials',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*rotor_mass # use interpolation for different nomial power
        materials[item] = item_mass
        
    for item in list(df.loc['Rotor Disposal'].index):
        split_ratio = list(df.loc['Rotor Disposal',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*rotor_mass
        disposal[item] = item_mass
        
    return materials,disposal  

def get_materials_nacelle(p,nacelle_mass): # get materials of nacelle
    rated_power = [30,150,600,800,2000]
    materials = {}
    disposal = {}
    
    for item in list(df.loc['Nacelle Materials'].index):
        split_ratio = list(df.loc['Nacelle Materials',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*nacelle_mass
        materials[item] = item_mass
        
    for item in list(df.loc['Nacelle Disposal'].index):
        split_ratio = list(df.loc['Nacelle Disposal',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*nacelle_mass
        disposal[item] = item_mass
    return materials,disposal

def get_materials_tower(p,tower_mass): # get materials of tower
    rated_power = [30,150,600,800,2000]
    materials = {}
    disposal = {}
    
    for item in list(df.loc['Tower Materials'].index):
        split_ratio = list(df.loc['Tower Materials',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*tower_mass
        materials[item] = item_mass
        
    for item in list(df.loc['Tower Disposal'].index):
        split_ratio = list(df.loc['Tower Disposal',item][:5])
        item_mass = np.interp(p,rated_power,split_ratio)*tower_mass
        disposal[item] = item_mass
        
    return materials,disposal

def get_materials_power_supply(p): # get materials of power supplies
    materials = {}
    disposal = {}
    
    rated_power = [30,150,600,800]
    total_mass = [617,1174,2068,2259]
    z = np.polyfit(rated_power,total_mass,1)
    mass_fit = np.poly1d(z)
    mass = mass_fit(p)
    
    if p > 800: # no data available for 2000 kW, thus an average value is applied
        for item in list(df.loc['Power Supply'].index)[1:]:
            materials[item]=mass*np.average(list(df.loc['Power Supply',item][:4]))
    else:
        for item in list(df.loc['Power Supply'].index)[1:]:
            rated_power_ = [30,150,600,800]
            materials[item]=mass*np.interp(p,rated_power_,list(df.loc['Power Supply',item][:4]))
    disposal['Copper'] = materials['Copper']
    disposal['Polymer'] = mass - materials['Copper']
    
    return mass,materials,disposal
          

In [4]:
# Transformers are from different datasets
# There are detailed data for some capacities
# The capacity of transformer for wind turbine is determined by its rated power

def get_materials_transformer(p,park_size):
    #Transformers standard sizes are 100, 160, 200, 315, 400, 500, 630, 1000, 1250, 1600, 2000, 2500 kVA for 33 kV.
    #Other sizes like 3000, 5000, 75000, 10000 kVA for large wind turbines
    #High voltage transformers for the whole wind farm, 250 MVA, 500 MVA if total capacity is greater than 250 MW
    materials = {}
    disposal = {}
    rated_capacity = [0,315,10000,16000,40000,63000,250000,500000]
    transformer_mass = [0,1477,27292,40983,62778,81500,197127,290868]
    
    material_name = ['Electric steel',
                    'Construction steel',
                    'Transformer oil',
                    'Aluminium',
                    'Insulation material',
                    'Porcelain',
                    'Copper',
                    'Paint',
                    'Wood'] # siliver is very little, so it's not considered, and some other unspecific parts
    
    average_split_ratio = [0.303949826,
                           0.234504934,
                           0.240013448,
                           0.021649156,
                           0.02678823,
                           0.005479727,
                           0.138699527,
                           0.00346364,
                           0.014397975]
    
    rated_capacity_ = [0,10000,16000,40000,63000,250000,500000] # there is no such data for 315 kVA transformer
    electricity_demand = [0,105200,207550,279000,302550,494000,750000]
    heat_demand = [0,19100,31800,47400,60030,162700,300000]
    
    mass = np.interp(p,rated_capacity,transformer_mass)
    for i in range(len(material_name)):
        materials[material_name[i]] = mass*average_split_ratio[i]
        
    materials['Electricity [kWh]'] = np.interp(p,rated_capacity_,electricity_demand)
    materials['Heat [kWh]'] = np.interp(p,rated_capacity_,heat_demand)
    
    if p*park_size > 30000: # if the total capacity of a wind farm is more than 250 MW (changed to 30 MW), high voltage transformer will be use to increase
                              # the voltage to 110 kV
        total_mass = 290868
        mass += total_mass
        materials_amount = [99640,53618,63000,0,6500,2650,8300,39960,2200,15000]
        i = 0
        for item in list(materials.keys()):
            if i < 10:
                materials[item] += materials_amount[i]/park_size
                i = i+1
        materials['Electricity [kWh]'] += 750000/park_size
        materials['Heat [kWh]'] += 300000/park_size
        
    disposal['Copper'] = materials['Copper']
    disposal['Steel'] = materials['Electric steel']+materials['Construction steel']
    disposal['Used oil'] = materials['Transformer oil']
    disposal['Aluminium'] = materials['Aluminium']
    disposal['Polymer'] = materials['Insulation material']+materials['Paint']
    disposal['Porcelain'] = materials['Porcelain']
        
    return mass,materials,disposal

In [5]:
# get materials and related land transformation or occupation
# Onshore and Offshore wind turbines have different foundation types
# Onshore foundations are made of steel and concrete

def get_materials_foundation(p,rotor_diameter,hub_height,Offshore = False):
    materials = {}
    disposal = {}
    if not(Offshore):
        rated_power = [30,150,600,800,2000]
        mass = 1696e3*hub_height/80*(rotor_diameter**2)/(100**2)/1.25 # derived from wind turbines with high hub height and capacity, 
                                            # the error is large for wind turbines with low hub height and capacity
                                            # but for future wind turbines the capacity will be large
                                            # over estimated sometimes, but haven't find a good solution
        i = 0
        for item in df.loc['Foundation'].index:
            if i < 2:
                materials[item] = mass*np.average(list(df.loc['Foundation',item][:4])) # average value
            if i >= 2 and i < len(df.loc['Foundation'].index):
                z = np.polyfit(rated_power,list(df.loc['Foundation',item][:5]),1) # regression
                fit = np.poly1d(z)
                materials[item] = fit(p)
            i = i+1
        materials['Concrete'] = materials['Concrete']/2200 # concrete density
        disposal['Steel'] = materials['Reinforcing steel']
        disposal['Concrete'] = mass - materials['Reinforcing steel']
                
    else: # Monopile is the main type foundation for offshore wind turbines, (70-80%)
           # This estimation is derived from wind turbines with a capacity from 3 MW to 10 MW
        rated_power = [0,3000,3600,4000,8000,10000]
        monopile_weight = [0,400000,475000,550000,725000,900000] # monopile and transition piece are made of steel
        transition_weight = [0,200000,200000,210000,285000,335000]
        grout_volume = [0,25,25,30,42.5,47.5]
        scour_protection_volume = [0,2150,2150,2550,3050,3550]
        materials['Steel'] = np.interp(p,rated_power,monopile_weight)+np.interp(p,rated_power,transition_weight)
        materials['Grout [m3]'] = np.interp(p,rated_power,grout_volume)
        materials['Scour protection [m3]'] = np.interp(p,rated_power,scour_protection_volume)
        mass = materials['Steel']+ materials['Grout [m3]']*(1500+1800)/2 # density of grout is typically 1500 to 1800 kg/m3
        
        disposal['Steel'] = materials['Steel']
        disposal['Grout'] = mass - materials['Steel']
    
    return mass,materials,disposal

In [6]:
def get_maintenance_act(p):
    maintenance_act = {}
    disposal = {}
    rated_power = [30,150,600,800,2000]
    oil_demand = [84,168,360,450,1000]
    z = np.polyfit(rated_power,oil_demand,1)
    oil_fit = np.poly1d(z)
    
    maintenance_act['Lubricating oil'] = oil_fit(p)
    disposal['Used oil'] = oil_fit(p)
    # transport for screw is calculated in get_transport_act()
    
    return maintenance_act,disposal

def cable_requirements(p,park_size,Offshore = False):
    # due to various types of the layout of wind farms, we use average values to estimate the distances
    # intra array payout
    # copper density is 8960 kg/m3, cross section of the cable is 95 mm2, its voltage is 33 kV, 
    # and max. current is 352 A, so it's okay for a wind turbine with a capacity less than 10 MW 
    # activities for laying the cable under the seabed didn't taken into consideration
    # although Aluminum conductors are also used, Copper is a better material for conductivity and high capacity
    rated_power = [30,150,600,800,2000]
    cable_requirements = {}
    disposal = {}
    distance_each = 2000 # transport distance for a wind turbine to the grid or the central transformer, 2 km
    distance_transformer = 20000 # for large wind farms, the distance from the central transformer to the grid, 20 km
    distance_coast = 5000 # average distance for an offshore wind turbine to shore, 5 km
    if Offshore:
        if p*park_size > 30000: # 250 MW might be too large, changed to 30 MW
            cross_sections = [400,500,630,800,1000,1200,1600,2000] # mm2
            rated_currents = [700,815,925,1045,1160,1335,1425,1560] # A
            total_power = [i*150 for i in rated_currents]
            copper_mass = (8960*95*1e-6*(distance_each+distance_coast)
                           +8960*np.interp(p*park_size,total_power,cross_sections)*1e-6*distance_transformer/park_size) 
                           # cables between transformer and grid should be 150 kV
        else:
            copper_mass = 8960*95*1e-6*(distance_each+distance_coast)
        insulation_mass = 617/220 * copper_mass
    if not(Offshore):
        if p*park_size > 30000:
            cross_sections = [400,500,630,800,1000,1200,1600,2000] # mm2
            rated_currents = [700,815,925,1045,1160,1335,1425,1560] # A
            total_power = [i*150 for i in rated_currents]
            copper_mass = (8960*95*1e-6*distance_each+8960*np.interp(p*park_size,total_power,cross_sections)*1e-6*distance_transformer/park_size)
        else:
            copper_mass = 8960*95*1e-6*distance_each
        insulation_mass = 617/220 * copper_mass
        
    total_mass = copper_mass + insulation_mass
    disposal['Copper'] = copper_mass
    disposal['Polymer'] = insulation_mass
    
    cable_requirements['Copper'] = copper_mass
    if p > 800: # no data available for 2000 kW, thus an average value is applied
        for item in list(df.loc['Power Supply'].index)[2:]:
            cable_requirements[item]=total_mass*np.average(list(df.loc['Power Supply',item][:4]))
    else:
        for item in list(df.loc['Power Supply'].index)[2:]:
            rated_power_ = [30,150,600,800]
            cable_requirements[item]=total_mass*np.interp(p,rated_power_,list(df.loc['Power Supply',item][:4]))
            
    return total_mass,cable_requirements,disposal
            
def get_transport_act(p,rotor_diameter,hub_height,park_size,rotor_mass,nacelle_mass,tower_mass,Offshore=False):
    foundation_mass = get_materials_foundation(p,rotor_diameter,hub_height,Offshore)[0]
    power_supply_mass = get_materials_power_supply(p)[0]
    transformer_mass = get_materials_transformer(p,park_size)[0]
    cable_mass = cable_requirements(p,park_size,Offshore)[0]
    transport_act = {}
    transport_by_truck = (600*rotor_mass+1025*nacelle_mass+1100*tower_mass
                          +50*foundation_mass+600*(power_supply_mass+transformer_mass+cable_mass))/1000
    transport_by_ship = 8050*tower_mass/1000
    transport_diposal = 200*(rotor_mass+nacelle_mass+tower_mass+power_supply_mass
                             +transformer_mass+cable_mass)/1000+50*foundation_mass/1000
    transport_maintenance = 2160/park_size
    transport_act['By truck'] = transport_by_truck+transport_diposal
    transport_act['By car'] = transport_maintenance
    transport_act['By ship'] = transport_by_ship
    return transport_act
    

In [7]:
cable_requirements(3000,30,Offshore = False)

(15567.185454545452,
 {'Copper': 4091.7333333333327,
  'HDPE granules': 4940.654697563742,
  'PP granules': 264.54618437449875,
  'PVC impact resistant': 3562.489640953581},
 {'Copper': 4091.7333333333327, 'Polymer': 11475.45212121212})

In [8]:
def get_assembly_act(p,park_size,rotor_mass,nacelle_mass,tower_mass,Offshore=False):
    # the electricity and heat for transformers are included in get_materials_transformer()
    assembly_act = {}
    rated_power = [30,150,600,800,2000]
    electricity_nacelle = [575,3987,17510,17510,67500]
    z1 = np.polyfit(rated_power,electricity_nacelle,2)
    electricity_fit = np.poly1d(z1)
    
    steel_arc = [84,115,152,190,228]
    z2 = np.polyfit(rated_power,steel_arc,1)
    steel_arc_fit = np.poly1d(z2)
    
    galvanizing = [74,190,625,818,1978]
    z3 = np.polyfit(rated_power,galvanizing,1)
    galvanizing_fit = np.poly1d(z3)
    
    diesel = [5381,5381,27360,29640,74500]
    z4 = np.polyfit(rated_power,diesel,1)
    diesel_fit = np.poly1d(z4)
    
    
    rotor_materials = get_materials_rotor(p,rotor_mass)[0]
    nacelle_materials = get_materials_nacelle(p,nacelle_mass)[0]
    tower_materials = get_materials_tower(p,tower_mass)[0]
    power_supply_materials = get_materials_power_supply(p)[1]
    transformer_materials = get_materials_transformer(p,park_size)[1]
    cable_materials = cable_requirements(p,park_size,Offshore)[1]
    
    assembly_act['market for wire drawing, copper'] = (nacelle_materials['Copper'] 
                                         + power_supply_materials['Copper']
                                         + transformer_materials['Copper']
                                         + cable_materials['Copper'])
    assembly_act['market for sheet rolling, aluminium'] = (nacelle_materials['Aluminium']
                                             + transformer_materials['Aluminium'])
    assembly_act['market for sheet rolling, chromium steel'] = (rotor_materials['Chromium steel'] 
                                                  + nacelle_materials['Chromium steel'])
    assembly_act['market for sheet rolling, steel'] = (rotor_materials['Cast iron']
                                         + nacelle_materials['Low-alloy steel']
                                         + nacelle_materials['Cast iron']
                                         + tower_materials['Low-alloy steel']
                                         + transformer_materials['Electric steel']
                                         + transformer_materials['Construction steel'])
    assembly_act['market for electricity, medium voltage'] = electricity_fit(p)+10
    assembly_act['market for welding, arc, steel'] = steel_arc_fit(p)
    assembly_act['market for zinc coat, pieces'] = galvanizing_fit(p)
    assembly_act['market for diesel, burned in building machine'] = diesel_fit(p)
    assembly_act['market for explosive, tovex'] = 10

    return assembly_act
    

In [9]:
def add_to_dict(dictionary, keys, values):
    for i in range(len(keys)):
        if keys[i] not in dictionary:
            dictionary[keys[i]] = values[i]
        else:
            dictionary[keys[i]] = dictionary[keys[i]] + values[i]

In [10]:
def get_disposal_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False):
    disposal_act = {}
    rotor_disposal = get_materials_rotor(p,rotor_mass)[1]
    nacelle_disposal = get_materials_nacelle(p,nacelle_mass)[1]
    tower_disposal = get_materials_tower(p,tower_mass)[1]
    foundation_disposal = get_materials_foundation(p,rotor_diameter,hub_height,Offshore)[2]
    power_supply_disposal = get_materials_power_supply(p)[2]
    transformer_disposal = get_materials_transformer(p,park_size)[2]
    cable_disposal = cable_requirements(p,park_size,Offshore)[2]
    maintenance_disposal = get_maintenance_act(p)[1]
    
    add_to_dict(disposal_act,list(rotor_disposal.keys()),list(rotor_disposal.values()))
    add_to_dict(disposal_act,list(nacelle_disposal.keys()),list(nacelle_disposal.values()))
    add_to_dict(disposal_act,list(tower_disposal.keys()),list(tower_disposal.values()))
    add_to_dict(disposal_act,list(foundation_disposal.keys()),list(foundation_disposal.values()))
    add_to_dict(disposal_act,list(power_supply_disposal.keys()),list(power_supply_disposal.values()))
    add_to_dict(disposal_act,list(transformer_disposal.keys()),list(transformer_disposal.values()))
    add_to_dict(disposal_act,list(cable_disposal.keys()),list(cable_disposal.values()))
    add_to_dict(disposal_act,list(maintenance_disposal.keys()),list(maintenance_disposal.values()))
    
    return disposal_act
    

In [33]:
def get_input_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False):
    
    rotor_materials = get_materials_rotor(p,rotor_mass)[0]
    print ('rotor',rotor_materials)
    nacelle_materials = get_materials_nacelle(p,nacelle_mass)[0]
    print ('nacelle',nacelle_materials)
    tower_materials = get_materials_tower(p,tower_mass)[0]
    print ('tower',tower_materials)
    foundation_materials = get_materials_foundation(p,rotor_diameter,hub_height,Offshore)[1]
    print ('foundation',foundation_materials)
    power_supply_materials = get_materials_power_supply(p)[1]
    print ('power_supply',power_supply_materials)
    transformer_materials = get_materials_transformer(p,park_size)[1]
    print ('transformer',transformer_materials)
    cable_materials = cable_requirements(p,park_size,Offshore)[1]
    print ('cable',cable_materials)
    maintenance_materials = get_maintenance_act(p)[0]
    print ('maintenance',maintenance_materials)
    
    
    input_materials = {}
    add_to_dict(input_materials,list(rotor_materials.keys()),list(rotor_materials.values())) # change to list
    add_to_dict(input_materials,list(nacelle_materials.keys()),list(nacelle_materials.values()))
    add_to_dict(input_materials,list(tower_materials.keys()),list(tower_materials.values()))
    add_to_dict(input_materials,list(foundation_materials.keys()),list(foundation_materials.values()))
    add_to_dict(input_materials,list(power_supply_materials.keys()),list(power_supply_materials.values()))
    add_to_dict(input_materials,list(transformer_materials.keys()),list(transformer_materials.values()))
    add_to_dict(input_materials,list(cable_materials.keys()),list(cable_materials.values()))
    
    activities = []
    materials = []
    for item in ['Rotor Materials','Nacelle Materials','Tower Materials','Foundation','Power Supply','Transformer']:
        if 'Total weight (kg)' not in list(df.loc[item].index):
            activities += list(df.loc[item]['Activity name'])
            materials += list(df.loc[item].index)
        else:
            activities += list(df.loc[item]['Activity name'][1:])
            materials += list(df.loc[item].index[1:]) 
    
    input_act = {}
    in_materials = []
    for activity,material in zip(activities,materials):
        if activity not in input_act:
            input_act[activity] = input_materials[material]
            in_materials.append(material)
        if activity in input_act and material not in in_materials:
            input_act[activity] += input_materials[material]
    input_act['market for electricity, medium voltage'] = input_materials['Electricity [kWh]']
    input_act['heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas'] = input_materials['Heat [kWh]']*3.6
                                                                                                                                    # convert kWh to MJ
    
    return input_materials,input_act
            
def get_LCI(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False):
    
    input_act = get_input_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore)[1]
    assembly_act = get_assembly_act(p,park_size,rotor_mass,nacelle_mass,tower_mass,Offshore)
    disposal_materials = get_disposal_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore)
    
    activities = []
    materials = []
    for item in ['Rotor Disposal',
                 'Nacelle Disposal',
                 'Tower Disposal',
                 'Foundation Disposal',
                 'Power Supply Disposal',
                 'Transformer Disposal']:
        activities += list(df.loc[item]['Activity name'])
        materials += list(df.loc[item].index) 
    
    disposal_act = {}
    disposal_act['market for iron scrap, unsorted'] = disposal_materials['Iron ']
    disposal_act['market for scrap steel'] = (disposal_materials['Steel']+disposal_materials['Low-alloy steel'])
    disposal_act['treatment of waste plastic, mixture, municipal incineration'] = (disposal_materials['Polymer']
                                                                                   +disposal_materials['Epoxy resin'])
    disposal_act['treatment of inert waste, inert material landfill'] = (disposal_materials['Glass']
                                                                         +disposal_materials['Concrete']
                                                                         +disposal_materials['Porcelain'])
    disposal_act['market for scrap aluminium'] = disposal_materials['Aluminium']
    disposal_act['market for scrap copper'] = disposal_materials['Copper']
    disposal_act['market for waste mineral oil'] = disposal_materials['Used oil']
            
    # transport
    transport_distance = get_transport_act(p,rotor_diameter,hub_height,park_size,rotor_mass,nacelle_mass,tower_mass,Offshore)
    transport_act = {}
    transport_act['market for transport, passenger car'] = transport_distance['By car']
    transport_act['market for transport, freight, lorry >32 metric ton, EURO6'] = transport_distance['By truck']
    transport_act['market for transport, freight, inland waterways, barge tanker'] = transport_distance['By ship']
    
    all_lci_act = {}
    add_to_dict(all_lci_act, list(input_act.keys()), list(input_act.values()))
    add_to_dict(all_lci_act, list(assembly_act.keys()), list(assembly_act.values()))
    add_to_dict(all_lci_act, list(transport_act.keys()), list(transport_act.values()))
    add_to_dict(all_lci_act, list(disposal_act.keys()), list(disposal_act.values()))
    
    return all_lci_act
    
    

In [34]:
get_input_materials(800,30,14700,20200,69735,50,50,Offshore=False)

rotor {'Fiberglass': 8400.0, 'Chromium steel': 3100.0, 'Cast iron': 3200.0}
nacelle {'Low-alloy steel': 3685.0, 'Chromium steel': 11426.000000000002, 'Cast iron': 3279.0, 'Rubber': 100.0, 'Aluminium': 207.0, 'Copper': 242.0, 'Fiberglass': 1261.0, 'Lubricating oil': 58.8}
tower {'Low-alloy steel': 69375.0, 'Epoxy resin': 360.0}
foundation {'Concrete': 91.40200375211009, 'Reinforcing steel': 10915.59174535777, 'Transformation [m^2]': 1121.3572996439198, 'Conversion (road) [m^2]': 999.9999999999997, 'Conversion (industrial) [m^2]': 121.35729964391976, 'Use of road [m^2]': 26609.2385737281, 'Use of industrial area [m^2]': 4602.41158967855}
power_supply {'Copper': 1275.3401109644396, 'HDPE granules': 622.4749596654701, 'PP granules': 20.95875285068923, 'PVC impact resistant': 448.51731100474956}
transformer {'Electric steel': 841.864755957307, 'Construction steel': 649.5198290809168, 'Transformer oil': 664.7770307557005, 'Aluminium': 59.96273027187609, 'Insulation material': 74.196675840433

({'Fiberglass': 9661.0,
  'Chromium steel': 14526.000000000002,
  'Cast iron': 6479.0,
  'Low-alloy steel': 73060.0,
  'Rubber': 100.0,
  'Aluminium': 266.96273027187607,
  'Copper': 3603.9030005633595,
  'Lubricating oil': 58.8,
  'Epoxy resin': 360.0,
  'Concrete': 91.40200375211009,
  'Reinforcing steel': 10915.59174535777,
  'Transformation [m^2]': 1121.3572996439198,
  'Conversion (road) [m^2]': 999.9999999999997,
  'Conversion (industrial) [m^2]': 121.35729964391976,
  'Use of road [m^2]': 26609.2385737281,
  'Use of industrial area [m^2]': 4602.41158967855,
  'HDPE granules': 2325.553206677422,
  'PP granules': 78.30145477028357,
  'PVC impact resistant': 1675.6511320840686,
  'Electric steel': 841.864755957307,
  'Construction steel': 649.5198290809168,
  'Transformer oil': 664.7770307557005,
  'Insulation material': 74.19667584043366,
  'Porcelain': 15.177468907541558,
  'Paint': 9.593413760743417,
  'Wood': 39.878778248270514,
  'Electricity [kWh]': 8416.0,
  'Heat [kWh]': 15

In [35]:
lci_act = get_LCI(800,30,14700,20200,69735,50,50,Offshore=False)
lci_act

rotor {'Fiberglass': 8400.0, 'Chromium steel': 3100.0, 'Cast iron': 3200.0}
nacelle {'Low-alloy steel': 3685.0, 'Chromium steel': 11426.000000000002, 'Cast iron': 3279.0, 'Rubber': 100.0, 'Aluminium': 207.0, 'Copper': 242.0, 'Fiberglass': 1261.0, 'Lubricating oil': 58.8}
tower {'Low-alloy steel': 69375.0, 'Epoxy resin': 360.0}
foundation {'Concrete': 91.40200375211009, 'Reinforcing steel': 10915.59174535777, 'Transformation [m^2]': 1121.3572996439198, 'Conversion (road) [m^2]': 999.9999999999997, 'Conversion (industrial) [m^2]': 121.35729964391976, 'Use of road [m^2]': 26609.2385737281, 'Use of industrial area [m^2]': 4602.41158967855}
power_supply {'Copper': 1275.3401109644396, 'HDPE granules': 622.4749596654701, 'PP granules': 20.95875285068923, 'PVC impact resistant': 448.51731100474956}
transformer {'Electric steel': 841.864755957307, 'Construction steel': 649.5198290809168, 'Transformer oil': 664.7770307557005, 'Aluminium': 59.96273027187609, 'Insulation material': 74.196675840433

{'market for glass fibre reinforced plastic, polyamide, injection moulded': 9661.0,
 'market for steel, chromium steel 18/8': 14526.000000000002,
 'market for cast iron': 6479.0,
 'market for steel, low-alloyed': 73060.0,
 'market for synthetic rubber': 100.0,
 'market for aluminium, wrought alloy': 266.96273027187607,
 'market for copper': 3603.9030005633595,
 'market for lubricating oil': 723.5770307557004,
 'market for epoxy resin, liquid': 360.0,
 'market for concrete, sole plate and foundation': 91.40200375211009,
 'market for reinforcing steel': 10915.59174535777,
 'Transformation, from pasture and meadow': 1121.3572996439198,
 'Transformation, to traffic area, road network': 999.9999999999997,
 'Transformation, to industrial area, built up': 121.35729964391976,
 'Occupation, traffic area, road network': 26609.2385737281,
 'Occupation, industrial area, built up': 4602.41158967855,
 'market for polyethylene, high density, granulate': 2325.553206677422,
 'market for polypropylene, 

In [36]:
#get_input_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False):
# 182 1.65 MW wind farm
get_input_materials(1650,182,42200,51000,136000,80,80,Offshore=False)

rotor {'Fiberglass': 24114.121474358974, 'Chromium steel': 8899.327548840049, 'Cast iron': 9186.550976800976}
nacelle {'Low-alloy steel': 9303.818441653191, 'Chromium steel': 26225.97528106892, 'Cast iron': 8346.260006693232, 'Rubber': 117.42755419933373, 'Aluminium': 522.4484765487264, 'Copper': 609.9643972767177, 'Fiberglass': 5874.1058425598785, 'Lubricating oil': 108.98291545741642}
tower {'Low-alloy steel': 135332.0063988221, 'Epoxy resin': 667.9936011779039}
foundation {'Concrete': 374.38260736864294, 'Reinforcing steel': 44710.26378898542, 'Transformation [m^2]': 1260.568069850251, 'Conversion (road) [m^2]': 999.9999999999994, 'Conversion (industrial) [m^2]': 260.56806985025116, 'Use of road [m^2]': 42893.200331691114, 'Use of industrial area [m^2]': 9799.671723330566}
power_supply {'Copper': 1802.0033338335568, 'HDPE granules': 1309.373170481218, 'PP granules': 70.11007596704796, 'PVC impact resistant': 944.1316265801846}
transformer {'Electric steel': 2077.9790020113005, 'Cons

({'Fiberglass': 29988.227316918852,
  'Chromium steel': 35125.30282990897,
  'Cast iron': 17532.810983494208,
  'Low-alloy steel': 144635.82484047528,
  'Rubber': 117.42755419933373,
  'Aluminium': 631.4604600801791,
  'Copper': 6827.609362837002,
  'Lubricating oil': 108.98291545741642,
  'Epoxy resin': 667.9936011779039,
  'Concrete': 374.38260736864294,
  'Reinforcing steel': 44710.26378898542,
  'Transformation [m^2]': 1260.568069850251,
  'Conversion (road) [m^2]': 999.9999999999994,
  'Conversion (industrial) [m^2]': 260.56806985025116,
  'Use of road [m^2]': 42893.200331691114,
  'Use of industrial area [m^2]': 9799.671723330566,
  'HDPE granules': 5742.765654771391,
  'PP granules': 307.495025401356,
  'PVC impact resistant': 4140.856709867881,
  'Electric steel': 2077.9790020113005,
  'Construction steel': 1475.428627424445,
  'Transformer oil': 1554.7156044368567,
  'Insulation material': 170.60352054620103,
  'Porcelain': 42.15301219841478,
  'Paint': 237.00122416372298,
  '

In [37]:
get_input_materials(2000,25,56350,72000,152000,110,80,Offshore=False)

rotor {'Fiberglass': 32199.690384615384, 'Chromium steel': 11883.348076923077, 'Cast iron': 12266.961538461537}
nacelle {'Low-alloy steel': 13134.86387548789, 'Chromium steel': 35500.78789788843, 'Cast iron': 11822.250236369366, 'Rubber': 87.2748430264976, 'Aluminium': 737.4724235739048, 'Copper': 860.5299522412664, 'Fiberglass': 9856.82077141264, 'Lubricating oil': 130.91226453974642}
tower {'Low-alloy steel': 151269.108714189, 'Epoxy resin': 730.8912858109832}
foundation {'Concrete': 707.8171170563405, 'Reinforcing steel': 84530.34247605057, 'Transformation [m^2]': 1317.8901516999167, 'Conversion (road) [m^2]': 999.9999999999994, 'Conversion (industrial) [m^2]': 317.890151699917, 'Use of road [m^2]': 49598.36105555824, 'Use of industrial area [m^2]': 11939.720013657867}
power_supply {'Copper': 2118.242267258302, 'HDPE granules': 1539.1589689386828, 'PP granules': 82.41390206429479, 'PVC impact resistant': 1109.8200984028817}
transformer {'Electric steel': 5799.664829248788, 'Construc

({'Fiberglass': 42056.51115602802,
  'Chromium steel': 47384.13597481151,
  'Cast iron': 24089.211774830903,
  'Low-alloy steel': 164403.97258967688,
  'Rubber': 87.2748430264976,
  'Aluminium': 866.6811584475363,
  'Copper': 8708.173100570455,
  'Lubricating oil': 130.91226453974642,
  'Epoxy resin': 730.8912858109832,
  'Concrete': 707.8171170563405,
  'Reinforcing steel': 84530.34247605057,
  'Transformation [m^2]': 1317.8901516999167,
  'Conversion (road) [m^2]': 999.9999999999994,
  'Conversion (industrial) [m^2]': 317.890151699917,
  'Use of road [m^2]': 49598.36105555824,
  'Use of industrial area [m^2]': 11939.720013657867,
  'HDPE granules': 7056.8244341011105,
  'PP granules': 377.8559911832606,
  'PVC impact resistant': 5088.366923701407,
  'Electric steel': 5799.664829248788,
  'Construction steel': 3544.3166329479272,
  'Transformer oil': 3952.473116676609,
  'Insulation material': 419.8802885342901,
  'Porcelain': 138.70467417403614,
  'Paint': 1619.0720549502119,
  'Wood

In [38]:
get_input_materials(3450,29,72000,108900,228660,94,100,Offshore=False)

rotor {'Fiberglass': 41142.46153846154, 'Chromium steel': 15183.692307692309, 'Cast iron': 15673.846153846152}
nacelle {'Low-alloy steel': 19866.481611675434, 'Chromium steel': 53694.941695556256, 'Cast iron': 17881.153482508667, 'Rubber': 132.00320007757765, 'Aluminium': 1115.427040655531, 'Copper': 1301.5515527649154, 'Fiberglass': 14908.441416761618, 'Lubricating oil': 198.00480011636645}
tower {'Low-alloy steel': 227560.48946438462, 'Epoxy resin': 1099.5105356153908}
foundation {'Concrete': 646.1024841229158, 'Reinforcing steel': 77160.13492958501, 'Transformation [m^2]': 1555.3673479342465, 'Conversion (road) [m^2]': 999.9999999999992, 'Conversion (industrial) [m^2]': 555.3673479342469, 'Use of road [m^2]': 77376.88405443633, 'Use of industrial area [m^2]': 20805.63435929954}
power_supply {'Copper': 3428.3749914465316, 'HDPE granules': 2491.1287054053223, 'PP granules': 133.3868958957459, 'PVC impact resistant': 1796.2437673826262}
transformer {'Electric steel': 6424.668653441513,

({'Fiberglass': 56050.90295522316,
  'Chromium steel': 68878.63400324856,
  'Cast iron': 33554.99963635482,
  'Low-alloy steel': 247426.97107606006,
  'Rubber': 132.00320007757765,
  'Aluminium': 1328.3080310896166,
  'Copper': 10554.121032793719,
  'Lubricating oil': 198.00480011636645,
  'Epoxy resin': 1099.5105356153908,
  'Concrete': 646.1024841229158,
  'Reinforcing steel': 77160.13492958501,
  'Transformation [m^2]': 1555.3673479342465,
  'Conversion (road) [m^2]': 999.9999999999992,
  'Conversion (industrial) [m^2]': 555.3673479342469,
  'Use of road [m^2]': 77376.88405443633,
  'Use of industrial area [m^2]': 20805.63435929954,
  'HDPE granules': 7531.2680180722855,
  'PP granules': 403.2599603986011,
  'PVC impact resistant': 5430.4674056370595,
  'Electric steel': 6424.668653441513,
  'Construction steel': 4154.835989344703,
  'Transformer oil': 4532.519680267822,
  'Insulation material': 487.5525849071828,
  'Porcelain': 145.2626908825447,
  'Paint': 1411.9897808702444,
  'W

In [39]:
#get_input_materials(p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False):
get_input_materials(3000,30,59400,98000,178000,90,94,Offshore=False)

rotor {'Fiberglass': 33942.53076923077, 'Chromium steel': 12526.546153846155, 'Cast iron': 12930.923076923076}
nacelle {'Low-alloy steel': 17878.009163858518, 'Chromium steel': 48320.51686101481, 'Cast iron': 16091.396155058304, 'Rubber': 118.79075856384397, 'Aluminium': 1003.7819098644816, 'Copper': 1171.2768794395015, 'Fiberglass': 13416.228272200537, 'Lubricating oil': 178.18613784576596}
tower {'Low-alloy steel': 177144.08783635293, 'Epoxy resin': 855.9121636470724}
foundation {'Concrete': 556.7478852548529, 'Reinforcing steel': 66489.05243932325, 'Transformation [m^2]': 1481.6675284132477, 'Conversion (road) [m^2]': 999.9999999999992, 'Conversion (industrial) [m^2]': 481.6675284132481, 'Use of road [m^2]': 68755.96312375003, 'Use of industrial area [m^2]': 18054.143700307297}
power_supply {'Copper': 3021.782077043288, 'HDPE granules': 2195.6898216742966, 'PP granules': 117.56769091357143, 'PVC impact resistant': 1583.215732182016}
transformer {'Electric steel': 5945.564890325023, 

({'Fiberglass': 47358.7590414313,
  'Chromium steel': 60847.06301486096,
  'Cast iron': 29022.319231981382,
  'Low-alloy steel': 195022.09700021145,
  'Rubber': 118.79075856384397,
  'Aluminium': 1190.695648572909,
  'Copper': 9758.958163780315,
  'Lubricating oil': 178.18613784576596,
  'Epoxy resin': 855.9121636470724,
  'Concrete': 556.7478852548529,
  'Reinforcing steel': 66489.05243932325,
  'Transformation [m^2]': 1481.6675284132477,
  'Conversion (road) [m^2]': 999.9999999999992,
  'Conversion (industrial) [m^2]': 481.6675284132481,
  'Use of road [m^2]': 68755.96312375003,
  'Use of industrial area [m^2]': 18054.143700307297,
  'HDPE granules': 7136.344519238039,
  'PP granules': 382.1138752880702,
  'PVC impact resistant': 5145.705373135597,
  'Electric steel': 5945.564890325023,
  'Construction steel': 3811.927302837103,
  'Transformer oil': 4172.219854944033,
  'Insulation material': 447.9499657791705,
  'Porcelain': 135.64401189611493,
  'Paint': 1361.9042559414352,
  'Wood

In [20]:
import brightway2 as bw

In [21]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.5 cutoff
	ecoinvent 3.6 cutoff

In [22]:
bw.methods

Methods dictionary with 850 objects, including:
	('CML 2001', 'acidification potential', 'average European')
	('CML 2001', 'acidification potential', 'generic')
	('CML 2001', 'climate change', 'GWP 100a')
	('CML 2001', 'climate change', 'GWP 20a')
	('CML 2001', 'climate change', 'GWP 500a')
	('CML 2001', 'climate change', 'lower limit of net GWP')
	('CML 2001', 'climate change', 'upper limit of net GWP')
	('CML 2001', 'eutrophication potential', 'average European')
	('CML 2001', 'eutrophication potential', 'generic')
	('CML 2001', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

In [23]:
eidb = bw.Database('ecoinvent 3.6 cutoff')

In [24]:
methods = ('IPCC 2013', 'climate change', 'GWP 100a')

In [81]:
#p,park_size,rotor_mass,nacelle_mass,tower_mass,rotor_diameter,hub_height,Offshore=False
lci_act = get_LCI(1650,182,42200,51000,136000,80,80,Offshore=False)
lci_act

rotor {'Fiberglass': 24114.121474358974, 'Chromium steel': 8899.327548840049, 'Cast iron': 9186.550976800976}
nacelle {'Low-alloy steel': 9303.818441653191, 'Chromium steel': 26225.97528106892, 'Cast iron': 8346.260006693232, 'Rubber': 117.42755419933373, 'Aluminium': 522.4484765487264, 'Copper': 609.9643972767177, 'Fiberglass': 5874.1058425598785, 'Lubricating oil': 108.98291545741642}
tower {'Low-alloy steel': 135332.0063988221, 'Epoxy resin': 667.9936011779039}
foundation {'Concrete': 374.38260736864294, 'Reinforcing steel': 44710.26378898542, 'Transformation [m^2]': 1260.568069850251, 'Conversion (road) [m^2]': 999.9999999999994, 'Conversion (industrial) [m^2]': 260.56806985025116, 'Use of road [m^2]': 42893.200331691114, 'Use of industrial area [m^2]': 9799.671723330566}
power_supply {'Copper': 1802.0033338335568, 'HDPE granules': 1309.373170481218, 'PP granules': 70.11007596704796, 'PVC impact resistant': 944.1316265801846}
transformer {'Electric steel': 2077.9790020113005, 'Cons

{'market for glass fibre reinforced plastic, polyamide, injection moulded': 29988.227316918852,
 'market for steel, chromium steel 18/8': 35125.30282990897,
 'market for cast iron': 17532.810983494208,
 'market for steel, low-alloyed': 144635.82484047528,
 'market for synthetic rubber': 117.42755419933373,
 'market for aluminium, wrought alloy': 631.4604600801791,
 'market for copper': 6827.609362837002,
 'market for lubricating oil': 1663.698519894273,
 'market for epoxy resin, liquid': 667.9936011779039,
 'market for concrete, sole plate and foundation': 374.38260736864294,
 'market for reinforcing steel': 44710.26378898542,
 'Transformation, from pasture and meadow': 1260.568069850251,
 'Transformation, to traffic area, road network': 999.9999999999994,
 'Transformation, to industrial area, built up': 260.56806985025116,
 'Occupation, traffic area, road network': 42893.200331691114,
 'Occupation, industrial area, built up': 9799.671723330566,
 'market for polyethylene, high density,

In [82]:
activities = list(lci_act.keys())
activities

['market for glass fibre reinforced plastic, polyamide, injection moulded',
 'market for steel, chromium steel 18/8',
 'market for cast iron',
 'market for steel, low-alloyed',
 'market for synthetic rubber',
 'market for aluminium, wrought alloy',
 'market for copper',
 'market for lubricating oil',
 'market for epoxy resin, liquid',
 'market for concrete, sole plate and foundation',
 'market for reinforcing steel',
 'Transformation, from pasture and meadow',
 'Transformation, to traffic area, road network',
 'Transformation, to industrial area, built up',
 'Occupation, traffic area, road network',
 'Occupation, industrial area, built up',
 'market for polyethylene, high density, granulate',
 'market for polypropylene, granulate',
 'market for polyvinylchloride, bulk polymerised',
 'steel production, electric, low-alloyed',
 'market for steel, unalloyed',
 'market for glass wool mat',
 'market for ceramic tile',
 'market for electrostatic paint',
 'planing, board, softwood, u=20%',
 '

In [83]:
act_eidb = {}
a = []
exception = ['market for electricity, medium voltage',
             'planing, board, softwood, u=20%',
             'treatment of waste plastic, mixture, municipal incineration'
             ]
for item in activities:
    lst = [act for act in eidb if act['name']==item]
    if len(lst) == 1:
        act_eidb[lst[0]] = lci_act[item]
        a.append(lst[0]['name'])
    if len(lst) > 1:
        for act in lst:
            if act['name'] in exception:
                continue
            else:
                if act['location'] == 'GLO':
                    act_eidb[act] = lci_act[item]
                    a.append(act['name'])
                    break
                else:
                    if act['location'] == 'RoW' and act['location'] != 'GLO':
                        act_eidb[act] = lci_act[item]
                        a.append(act['name'])
                        break
act_eidb

{'market for glass fibre reinforced plastic, polyamide, injection moulded' (kilogram, GLO, None): 29988.227316918852,
 'market for steel, chromium steel 18/8' (kilogram, GLO, None): 35125.30282990897,
 'market for cast iron' (kilogram, GLO, None): 17532.810983494208,
 'market for steel, low-alloyed' (kilogram, GLO, None): 144635.82484047528,
 'market for synthetic rubber' (kilogram, GLO, None): 117.42755419933373,
 'market for aluminium, wrought alloy' (kilogram, GLO, None): 631.4604600801791,
 'market for copper' (kilogram, GLO, None): 6827.609362837002,
 'market for lubricating oil' (kilogram, RoW, None): 1663.698519894273,
 'market for epoxy resin, liquid' (kilogram, RoW, None): 667.9936011779039,
 'market for concrete, sole plate and foundation' (cubic meter, CH, None): 374.38260736864294,
 'market for reinforcing steel' (kilogram, GLO, None): 44710.26378898542,
 'market for polyethylene, high density, granulate' (kilogram, GLO, None): 5742.765654771391,
 'market for polypropylene,

In [84]:
len(activities),len(act_eidb)

(45, 37)

In [85]:
act = [act for act in eidb if act['name'] == 'market for electricity, medium voltage']
act_eidb[act[0]] = lci_act['market for electricity, medium voltage']
act_wood = [act for act in eidb if act['name'] == 'planing, board, softwood, u=20%' 
                                and act['unit'] == 'kilogram' 
                                and act['location'] == 'RoW'][0]
act_plastic = [act for act in eidb if act['name'] == 'treatment of waste plastic, mixture, municipal incineration' 
                                   and act['unit'] == 'kilogram'
                                   and act['location'] == 'RoW'][0]
act_eidb[act_wood] = lci_act['planing, board, softwood, u=20%']
act_eidb[act_plastic] = lci_act['treatment of waste plastic, mixture, municipal incineration']
act_eidb

{'market for glass fibre reinforced plastic, polyamide, injection moulded' (kilogram, GLO, None): 29988.227316918852,
 'market for steel, chromium steel 18/8' (kilogram, GLO, None): 35125.30282990897,
 'market for cast iron' (kilogram, GLO, None): 17532.810983494208,
 'market for steel, low-alloyed' (kilogram, GLO, None): 144635.82484047528,
 'market for synthetic rubber' (kilogram, GLO, None): 117.42755419933373,
 'market for aluminium, wrought alloy' (kilogram, GLO, None): 631.4604600801791,
 'market for copper' (kilogram, GLO, None): 6827.609362837002,
 'market for lubricating oil' (kilogram, RoW, None): 1663.698519894273,
 'market for epoxy resin, liquid' (kilogram, RoW, None): 667.9936011779039,
 'market for concrete, sole plate and foundation' (cubic meter, CH, None): 374.38260736864294,
 'market for reinforcing steel' (kilogram, GLO, None): 44710.26378898542,
 'market for polyethylene, high density, granulate' (kilogram, GLO, None): 5742.765654771391,
 'market for polypropylene,

In [86]:
len(activities),len(act_eidb)

(45, 40)

In [87]:
methods = [('IPCC 2013', 'climate change', 'GWP 100a'),
           ('cumulative energy demand', 'fossil', 'non-renewable energy resources, fossil')]

In [88]:
bw.calculation_setups['multiLCA'] = {'inv': [act_eidb], 'ia': methods}
bw.calculation_setups['multiLCA']
myMultiLCA = bw.MultiLCA('multiLCA')
myMultiLCA.results
df_impact = pd.DataFrame(columns = methods, data = myMultiLCA.results)
df_impact

,"(IPCC 2013, climate change, GWP 100a)","(cumulative energy demand, fossil, non-renewable energy resources, fossil)"
0,1.119734e+06,1.379661e+07
